# Данные для датасетов

Подключаем Гугл Диск

In [5]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Bing

https://github.com/ultralytics/google-images-download

Параметры https://google-images-download.readthedocs.io/en/latest/arguments.html

### Установка

In [4]:
!apt-get update > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1
!pip install -U selenium
%cd /content/
!git clone https://github.com/ultralytics/google-images-download bing

     |████████████████████████████████| 911kB 5.0MB/s 
/content
Cloning into 'bing'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 729 (delta 4), reused 11 (delta 3), pack-reused 715
Receiving objects: 100% (729/729), 313.34 KiB | 11.60 MiB/s, done.
Resolving deltas: 100% (404/404), done.


In [ ]:
!mkdir -p /content/drive/My\ Drive/stylegan/scrape/bing

**Параметры**

`search` - поисковый запрос

`limit` - ограничение на количество скачиваемых файлов

In [6]:
%cd /content/drive/My\ Drive/stylegan/scrape/bing
!python /content/bing/bing_scraper.py --search 'monty python' --limit 10 --download --chromedriver 'chromedriver'

/content/drive/My Drive/stylegan/scrape/bing
Searching for https://www.bing.com/images/search?q=monty%20python
1/10 https://www.syfy.com/sites/syfy/files/styles/1200x680/public/2018/03/monty-python.jpg 
2/10 http://www.liveforfilm.com/wp-content/uploads/2019/06/monty-python.jpg 
3/10 https://d13ezvd6yrslxm.cloudfront.net/wp/wp-content/images/Monty-Pythons-Flying-Circus.jpg 
4/10 https://secure.i.telegraph.co.uk/multimedia/archive/02757/monty_python_2757869k.jpg 
5/10 https://assets.nst.com.my/images/articles/MONTYPYTHON-AmonNNIVERSARY__1570289712.jpg 
6/10 https://images2.minutemediacdn.com/image/upload/c_fill,g_auto,h_1248,w_2220/v1555434576/shape/mentalfloss/python_lead.jpg 
7/10 https://pixel.nymag.com/imgs/daily/vulture/2018/08/07/07-monty-python.w700.h700.jpg 
8/10 URLError on an image...trying next one... Error: HTTP Error 401: Unauthorized - missing signature
8/10 http://highlighthollywood.com/wp-content/uploads/2020/01/monty-python-terry-jones-dies-scaled.jpg 
9/10 https://stat

## Обработка

In [ ]:
import os

input_dir = '/content/drive/My Drive/stylegan/scrape/bing/images/robots' 
processed_dir = '/content/drive/My Drive/stylegan/data/robots' 
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

Запускаем функцию, которая удалит все файлы, которые нейросеть не может загрузить

In [ ]:
from fastai.vision import verify_images
verify_images(processed_dir, delete=True)

Добавляем функции:

`resize` - изменяет размер изображений в папке `in_dir` на `res` (по умолчанию 1024х1024) и сохраняет измененные в папку `out_dir`

`crop` - обрезает изображения в папке `in_dir` до квадрата, меняет размер на `res` (по умолчанию 1024х1024) и сохраняет измененные в папку `out_dir`

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

def resize(in_dir, out_dir, res=1024):
    files = os.listdir(in_dir)
    for i in tqdm(range(len(files))):
      if 'DS_Store' in files[i]:
        continue
      im = Image.open(f"{in_dir}/{files[i]}")
      imResize = im.resize((res,res), Image.ANTIALIAS)
      imResize.save(f'{out_dir}/{i:6d}.png' , 'PNG')

def crop(in_dir, out_dir, res=1024):
    files = os.listdir(in_dir)
    for i in tqdm(range(len(files))):
      if 'DS_Store' in files[i]:
        continue
      im = Image.open(f"{in_dir}/{files[i]}")
      width, height = im.size
      new_res = min(width, height)
      left = (width - new_res)/2
      top = (height - new_res)/2
      right = (width + new_res)/2
      bottom = (height + new_res)/2

      im = im.crop((left, top, right, bottom))
      imResize = im.resize((res,res), Image.ANTIALIAS)
      imResize.save(f'{out_dir}/{i:6d}.png' , 'PNG')

In [ ]:
crop(input_dir, processed_dir, 256)

## Более хитрый кроп

1. Изменяет размер изображения до нужного разрешения по меньшей стороне

2. Разрезает его на квадратные изображения

3. Может аугментировать датасет (дополнительно сохранять перевернутые и отзеркаленые изображения)

In [7]:
import glob
import os
from math import floor, ceil
from PIL import Image
from tqdm import tqdm

**Вспомогательные функции**

In [1]:
def thumbnail(img, size=256):
    """
    Изменяет размер изображения так, чтобы меньшая его сторона равнялась 
    значению переменной size
    """
    
    img = img.copy()

    if img.mode not in ('L', 'RGB'):
        img = img.convert('RGB')

    width, height = img.size

    if width == height:
        img.thumbnail((size, size), Image.ANTIALIAS)

    elif height > width:
        ratio = float(height) / float(width)
        newheight = ratio * size
        img = img.resize((size, int(floor(newheight))), Image.ANTIALIAS)

    elif width > height:
        ratio = float(width) / float(height)
        newwidth = ratio * size
        img = img.resize((int(floor(newwidth)), size), Image.ANTIALIAS)


    return img

In [3]:
def get_crop_bboxes(w, h):
    """
    На основе отношения ширины к высоте, вычисляет на какое количество квадратов
    надо разрезать изображение, а так же границы этих квадратов
    """
    n_step = ceil(float(w) / float(h))
    shift = (w - h) // n_step
    shifting_array = []
    for step in range(0, n_step):
        if w > h:
            shifting = (shift * step, 0, shift * step + h, h)
        else:
            shifting = (0, shift * step, w, shift * step + w)
        shifting_array.append(shifting)
    if w > h:
        shifting = (w - h, 0, w, h)
    else:
        shifting = (0, h - w, w, h)
    shifting_array.append(shifting)
    return shifting_array

In [4]:
def save_cuts(img, cnt, path, augment=False):
    """
    Сохраняет вырезанное изображение в папку path под номером cnt
    Если аргумент augment равен True также поворачивает это изображение
    под разными углами, отзеркаливает его и сохраняет в ту же папку
    Возвращает новое значение cnt
    """
    img = img.copy()

    img.save(f'{path}/{cnt:08d}.png')
    cnt += 1
    if augment:
        img.transpose(Image.FLIP_LEFT_RIGHT).save(f'{path}/{cnt:08d}.png')
        cnt += 1
        img.transpose(Image.FLIP_TOP_BOTTOM).save(f'{path}/{cnt:08d}.png')
        cnt += 1
        img.transpose(Image.ROTATE_180).save(f'{path}/{cnt:08d}.png')
        cnt += 1

    return cnt

**Параметры**

In [8]:
input_dir = '/content/drive/My Drive/stylegan/scrape/bing/images/space' 
processed_dir = '/content/drive/My Drive/stylegan/data/space2' 
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

images = glob.glob(f'{input_dir}/*.png')+glob.glob(f'{input_dir}/*.jpg')
total = len(images)
print(f'Number of images: {total}')

img_size =1024
img_cnt = 0

Number of images: 99


In [ ]:
for i, image in enumerate(tqdm(images, total=total)):
    img = Image.open(image)
    img = thumbnail(img, size=img_size)
    w, h = img.size
    bboxes = get_crop_bboxes(w, h)
    for bbox in bboxes:
        img_cropped = img.crop(bbox)
        img_cnt = save_cuts(img_cropped, img_cnt, processed_dir, False)

## Определение и выравнивание лиц

Скачиваем веса для определения лиц

In [ ]:
%cd /content/
!wget https://raw.githubusercontent.com/artem-konevskikh/da-workshop/master/weights/shape_predictor_68_face_landmarks.dat

In [10]:
import cv2
import glob
import os
import uuid
import dlib

In [19]:
PREDICTOR_PATH = "/content/shape_predictor_68_face_landmarks.dat"
FACES_FOLDER = "/content/drive/My Drive/stylegan/scrape/bing/images/monty_python"
OUTPUT_FOLDER = "/content/drive/My Drive/stylegan/data/monty_python/"
if not os.path.exists(OUTPUT_FOLDER):
  os.makedirs(OUTPUT_FOLDER)



In [16]:
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

Запускаем обработку

In [ ]:
for f in glob.glob(os.path.join(FACES_FOLDER, "*.jpg")):
    # Load the image using Dlib
    try:
        img = dlib.load_rgb_image(f)
    except:
        continue
    print("Processing file: {}".format(f))
    dets = detector(img, 1)

    num_faces = len(dets)
    print(f"Faces found: {num_faces}")
    if num_faces == 0:
        print(f"Sorry, there were no faces found in '{f}'")
        continue

    faces = dlib.full_object_detections()
    for detection in dets:
        faces.append(sp(img, detection))

 
    images = dlib.get_face_chips(img, faces, size=256, padding=0.5)

    for image in images:
        dlib.save_image(image, OUTPUT_FOLDER + str(uuid.uuid4()) + ".jpg")